In [20]:
from cognitive.task_bank import *
from cognitive import stim_generator as sg
from cognitive import task_generator as tg
import cognitive.info_generator as ig
import os
from PIL import Image
import json

In [11]:
"""
class SequentialCategoryMatch(TemporalTask):
    def __init__(self, whens=None, first_shareable=None, n_frames=1):
        super(SequentialCategoryMatch, self).__init__(whens=whens, first_shareable=first_shareable)
        total_frames = n_frames * 2 + random.randint(0, const.MAX_MEMORY - (n_frames * 2) + 1)

        sample_objs = [tg.Select(when=f'last{total_frames - i - 1}') for i in range(n_frames)]
        response_objs = [tg.Select(when=f'last{i}') for i in range(n_frames)]
        sample_cats = [tg.GetCategory(obj) for obj in sample_objs]
        response_cats = [tg.GetCategory(obj) for obj in response_objs]
        is_sames = [tg.IsSame(sample, response) for sample, response in zip(sample_cats, response_cats)]
        if n_frames == 1:
            self._operator = is_sames[0]
        else:
            ands = tg.And(is_sames[0], is_sames[1])
            for is_same1, is_same2 in zip(is_sames[2::2], is_sames[3::2]):
                ands = tg.And(tg.And(is_same1, is_same2), ands)
            self._operator = ands
        self.n_frames = total_frames

    @property
    def instance_size(self):
        return sg.n_sample_shape(2) * (sg.n_random_when()) ** 2

"""

"\nclass SequentialCategoryMatch(TemporalTask):\n    def __init__(self, whens=None, first_shareable=None, n_frames=1):\n        super(SequentialCategoryMatch, self).__init__(whens=whens, first_shareable=first_shareable)\n        total_frames = n_frames * 2 + random.randint(0, const.MAX_MEMORY - (n_frames * 2) + 1)\n\n        sample_objs = [tg.Select(when=f'last{total_frames - i - 1}') for i in range(n_frames)]\n        response_objs = [tg.Select(when=f'last{i}') for i in range(n_frames)]\n        sample_cats = [tg.GetCategory(obj) for obj in sample_objs]\n        response_cats = [tg.GetCategory(obj) for obj in response_objs]\n        is_sames = [tg.IsSame(sample, response) for sample, response in zip(sample_cats, response_cats)]\n        if n_frames == 1:\n            self._operator = is_sames[0]\n        else:\n            ands = tg.And(is_sames[0], is_sames[1])\n            for is_same1, is_same2 in zip(is_sames[2::2], is_sames[3::2]):\n                ands = tg.And(tg.And(is_same1, 

# Initialize Parameters

In [21]:
output_dir='./data'
stim_dir='./data/MULTIF_5_stim'
random_families=True
families=['CompareLoc']
composition=3
temporal_switch=0
nback = 1
nback_length = 6
seq_length = 6
seq_reverse = False
fix_delay = False
fixation_cue = True



const.DATA = const.Data(stim_dir)
assert all('Compare' in family for family in families)
composition = nback_length - nback + 1

output_dir=output_dir
composition=composition
img_size=224
first_shareable=1
temporal_switch=0
max_memory=12


if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Stimuli Directory:  ./data/MULTIF_5_stim
OrderedDict([((0, 0), [(0.0, 0.25), (0.0, 0.25)]), ((0, 1), [(0.0, 0.25), (0.25, 0.5)]), ((0, 2), [(0.0, 0.25), (0.5, 0.75)]), ((0, 3), [(0.0, 0.25), (0.75, 1.0)]), ((1, 0), [(0.25, 0.5), (0.0, 0.25)]), ((1, 1), [(0.25, 0.5), (0.25, 0.5)]), ((1, 2), [(0.25, 0.5), (0.5, 0.75)]), ((1, 3), [(0.25, 0.5), (0.75, 1.0)]), ((2, 0), [(0.5, 0.75), (0.0, 0.25)]), ((2, 1), [(0.5, 0.75), (0.25, 0.5)]), ((2, 2), [(0.5, 0.75), (0.5, 0.75)]), ((2, 3), [(0.5, 0.75), (0.75, 1.0)]), ((3, 0), [(0.75, 1.0), (0.0, 0.25)]), ((3, 1), [(0.75, 1.0), (0.25, 0.5)]), ((3, 2), [(0.75, 1.0), (0.5, 0.75)]), ((3, 3), [(0.75, 1.0), (0.75, 1.0)])])


In [22]:
def generate_temporal_example(whens, first_shareable=None, temporal_switch=temporal_switch):
    # max_memory


    task = CompareLocTemporal( whens, first_shareable)
    assert isinstance(task, tg.TemporalTask)

    # To get maximum memory duration, we need to specify the following average
    # memory value
    avg_mem = round(max_memory / 3.0 + 0.01, 2)

    objset = task.generate_objset(n_distractor=0,average_memory_span=avg_mem, temporal_switch=temporal_switch)
   

    # Getting targets can remove some objects from objset.
    # Create example fields after this call.
    frame_info = ig.FrameInfo(task, objset)
    compo_info = ig.TaskInfoCompo(task, frame_info)
    return compo_info

In [23]:
def generate_compo_temporal_example(n_tasks=1,
                                    whens=None, temporal_switch=temporal_switch):
    '''

    :param first_shareable:
    :param whens:
    :param families:
    :param max_memory:
    :param max_distractors:
    :param n_tasks:
    :return: combined TaskInfo Compo
    '''
    if n_tasks == 1:
        return generate_temporal_example(whens=whens, first_shareable=1, temporal_switch=temporal_switch)
    else:
        compo_tasks = [generate_temporal_example(whens=whens, first_shareable=1, temporal_switch=temporal_switch)
                    for _ in range(n_tasks)]
        # temporal combination
        cur_task = compo_tasks[0]
        for task in compo_tasks[1:]:
            cur_task.merge(task)
        return cur_task

In [24]:
task_info = generate_compo_temporal_example(n_tasks=3, whens=[f'last{nback}', 'last0'], temporal_switch=temporal_switch)

In [25]:
def write_task_instance(fname, task_info, img_size, fixation_cue=True):
    if not os.path.exists(fname):
        os.makedirs(fname)
    objset = task_info.frame_info.objset
    for i, (epoch, frame) in enumerate(zip(sg.render(objset, img_size), task_info.frame_info)):
        if fixation_cue:
            if not any('ending' in description for description in frame.description):
                sg.add_fixation_cue(epoch)
        img = Image.fromarray(epoch, 'RGB')
        filename = os.path.join(fname, f'epoch{i}.png')
        img.save(filename)

    examples, compo_example, memory_info = task_info.get_examples()
    for i, task_example in enumerate(examples):
        filename = os.path.join(fname, f'task{i} example')
        with open(filename, 'w') as f:
            json.dump(task_example, f, indent=4)

    filename = os.path.join(fname, 'compo_task_example')
    with open(filename, 'w') as f:
        json.dump(compo_example, f, indent=4)

    filename = os.path.join(fname, 'memory_trace_info')
    with open(filename, 'w') as f:
        json.dump(memory_info, f, indent=4)

    filename = os.path.join(fname, 'frame_info')
    with open(filename, 'w') as f:
        json.dump(task_info.frame_info.dump(), f, indent=4)

In [26]:
write_task_instance('test_test', task_info, img_size, fixation_cue)

In [18]:
def write_trial_instance(task: tg.TemporalTask, write_fp: str, img_size=224, fixation_cue=True) -> None:
    # save the actual generated frames into another folder
    if os.path.exists(write_fp):
        shutil.rmtree(write_fp)
    os.makedirs(write_fp)
    frame_info = ig.FrameInfo(task, task.generate_objset())
    compo_info = ig.TaskInfoCompo(task, frame_info)
    objset = compo_info.frame_info.objset
    for i, (epoch, frame) in enumerate(zip(sg.render(objset, img_size), compo_info.frame_info)):
        # add cross in the center of the image
        if fixation_cue:
            if not any('ending' in description for description in frame.description):
                sg.add_fixation_cue(epoch)
        img = Image.fromarray(epoch, 'RGB') # just store rgb
        filename = os.path.join(write_fp, f'epoch{i}.png')
        img.save(filename)
    _, compo_example, _ = compo_info.get_examples()
    filename = os.path.join(write_fp, 'trial_info')
    with open(filename, 'w') as f:
        json.dump(compo_example, f, indent=4)
    return